# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [1]:
#imports
import numpy as np
import pandas as pd

#load data
recipes = pd.read_csv("recipes.tsv", sep='\t') #contains recipes
recipe_tags = pd.read_csv("recipe-tags.tsv", sep='\t') #contains tags
#create helper to add features (following class example) -> checking if recipe has certain ingredient
def add_feature(feature_name):
    recipes[feature_name] = recipes['recipe_title'].str.lower().str.contains(feature_name.lower())

##### Preferences picked:
* likes: mushroom, chicken, almond, and peach
    - My thought process behind this was thinking of some of my favorite recipes. I am a big fan of any dish that has mushrooms in it. I also couldn't help but think of my dog when thinking of ingredients and she loves chicken so I threw that in too! Almond and peach came from thinking of some of my favorite desserts (almond croissants and coookies, peach cobbler, etc.)
* dislikes: oyster, chocolate, and peanut
    - I am allergic to seafood, so I looked through the "recipes.tsv" file to see if there were any seafood recipes and picked one to exclude. I am not too big of a fan of chocolate; I find it overpowering unless in small amounts, so I decided to rate it lower. I have a friend who is not a bg fan of peanut dishes, so I decided to rate this low as well!

In [2]:
#add specific ingredient fts from preferences 
prefs = ["oyster", "mushroom", "chicken", "chocolate", "almond", "peanut", "peach"]
for item in prefs:
    add_feature(item)
#initialize raw ratings (following class example)
ratings = pd.DataFrame({"recipe_title": recipes["recipe_title"],
                               "rating_raw": np.random.uniform(low=2, high=3, size=len(recipes))},
                              index=recipes.index)
#adjust ratings based on preferences
#set preferences 
ratings.loc[recipes['oyster'], 'rating_raw'] = 1
ratings.loc[recipes['mushroom'], 'rating_raw'] = 5 
ratings.loc[recipes['chicken'], 'rating_raw'] = 5
ratings.loc[recipes['chocolate'], 'rating_raw'] = 1
ratings.loc[recipes['almond'], 'rating_raw'] = 5
ratings.loc[recipes['peanut'], 'rating_raw'] = 1
ratings.loc[recipes['peach'], 'rating_raw'] = 5

#normalize ratings for [0,1] scale, with 0 = worst, 1 = best
ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25

#check that there are at least 10 recipes with rating 1
assert len(ratings[ratings['rating'] ==1]) >=10, "Less than 10 recipes w/rating 1"
#check there are at least 10 recipes with rating 0
assert len(ratings[ratings['rating'] == 0]) >= 10, "Less than 10 recipes w/rating 0"

#save ratings in df w/both 'rating' and 'recipe_slug' in tsv file:
ratings_df = pd.DataFrame({"recipe_slug": recipes["recipe_slug"],
                           "rating": ratings["rating"]})
ratings_df.head(3)

,recipe_slug,rating
0,falafel,0.253723
1,spamburger,0.337474
2,bacon-fried-rice,0.275984


In [3]:
#convert to tsv 
ratings_df.to_csv("ratings.tsv", sep="\t", index=False)

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [4]:
# YOUR CHANGES HERE
#already have recipe_slug col. from recipes df
#col. for each tag that appears in reipe-tags.tsv; 0-1 encoding of tags
#first, one-hot encode tags
rtags_dummies = pd.get_dummies(recipe_tags['recipe_tag'])

#combine encoded w/ recipe_slug
encoded = pd.concat([recipe_tags[['recipe_slug']], rtags_dummies], axis=1)

#group tags by recipe_slug
features_df = encoded.groupby('recipe_slug').max().reset_index() #use max to ensure 0/1 encoding for recipes w multiple tags

#create col. bias hard-coded to one
features_df.insert(1, 'bias', 1)

#convert bool to integers (0/1)
feature_cols = features_df.columns.drop("recipe_slug")
features_df[feature_cols] = features_df[feature_cols].astype(int)
features_df.head(3)

,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,almond-chip-cookies,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,almond-croissants,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
#convert to tsv
features_df.to_csv("features.tsv", sep='\t', index=False)

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [6]:
# YOUR CHANGES HERE
#more imports
from sklearn.linear_model import Ridge

#combine features, ratings on recipe_slug
df = pd.merge(features_df, ratings_df, on='recipe_slug')

#sep. target variable (rating)
y = df['rating']
X = df.drop(columns=['recipe_slug', 'rating']) #gives tags and bias

#fit ridge reg.
model = Ridge(alpha=1, fit_intercept=False)
model.fit(X, y)

#save coefs
coef_df = pd.DataFrame({
    "recipe_tag": X.columns,
    "coefficient": model.coef_
})
coef_df.head(3)

,recipe_tag,coefficient
0,bias,0.317304
1,alfredo,0.070769
2,almond,0.275765


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [7]:
#save coefs in tsv
coef_df.to_csv("model.tsv", sep='\t', index=False)

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [8]:
# YOUR CHANGES HERE
#extract coefficients 
b = coef_df['coefficient'].values #learned weights for bias and tags

#compute predicted scores: score_est = X @ b; lin reg: X^T * beta_hat
score_est = X.values @ b

#store estimates in df
estimates_df = pd.DataFrame({
    "recipe_slug": df['recipe_slug'],
    "score_estimate": score_est
})
estimates_df.head(3)

,recipe_slug,score_estimate
0,almond-chip-cookies,0.837669
1,almond-croissants,0.852390
2,apple-crisp,0.410054


Submit "estimates.tsv" in Gradescope.

In [9]:
#save as tsv
estimates_df.to_csv("estimates.tsv", sep='\t', index=False)

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [10]:
# YOUR CHANGES HERE
#LinUCB formula: UCB(x) = x^T * beta_hat + alpha * sqrt(x^T * inv(A) * x)
#where 
    #A = X^T * X + lambda_reg * I
    #beta_hat = inv(A)*X^T*y
#params (from instuctions)
lambda_reg = 1
alpha = 2

#convert X df to numpy array for calculations
X_np = X.values
n, d = X_np.shape #number of recipes and # of features

#calculate A and its inverse
A = X_np.T @ X_np + lambda_reg * np.eye(d)
A_inv = np.linalg.inv(A)

#calculate beta hat (ridge estimate)
beta_hat = A_inv @ X_np.T @ y

#calculate variance terms -> var_term = sqrt(x^T * inv(A) * x) for each recipe
var_terms = np.sqrt(np.sum((X_np @ A_inv) * X_np, axis=1))

#calculate upper bounds
bounds = X_np @ beta_hat + alpha * var_terms

#save as df
bounds_df = pd.DataFrame({
    "recipe_slug": df["recipe_slug"],
    "score_bound": bounds
})
bounds_df.head(3)

,recipe_slug,score_bound
0,almond-chip-cookies,2.561127
1,almond-croissants,2.558246
2,apple-crisp,2.186395


Submit "bounds.tsv" in Gradescope.

In [11]:
#convert to tsv
bounds_df.to_csv("bounds.tsv", sep='\t', index=False)

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

Hint: do not remove recipes after each recommendation.
Repeating recommendations is expected.

In [12]:
# YOUR CHANGES HERE
recipe_slugs = df['recipe_slug'].values 
true_rewards = df["rating"].astype(float).values #actual ratings from ratings.tsv 

#LinUCB params
lambda_reg = 1 #same as p5
alpha = 2 #same as p5
T = 100

#initialize LinUCB matrices
A = lambda_reg * np.eye(d) #cov. matrix
b = np.zeros(d) #reward-weighted feature sum

#store recommendations
recs = []

#create loop for online recommmendation
for t in range(T):
    #compute current theta estimate
    A_inv = np.linalg.inv(A)
    theta_hat = A_inv @ b

    #compute UCB for all recipes
    est = X_np @ theta_hat
    var = np.sqrt(np.sum((X_np @ A_inv) * X_np, axis=1))
    ucb = est + alpha * var

    #pick recipes w/highest UCB
    chosen_index = np.argmax(ucb)
    chosen_slug = recipe_slugs[chosen_index]
    chosen_bound = ucb[chosen_index]
    reward = true_rewards[chosen_index] #observe reward

    #store results in recs
    recs.append((chosen_slug, chosen_bound, reward))

    #update matrices w/ observed reward
    x = X_np[chosen_index] #ft. vector of chosen recipe
    A += np.outer(x,x) #covaariance
    b += reward*x #weighted rewards

#save in df
recs_df = pd.DataFrame(recs, columns=['recipe_slug', 'score_bound', 'reward'])
recs_df.head(3)

,recipe_slug,score_bound,reward
0,apple-crumble,7.483315,0.407673
1,ma-la-chicken,7.219767,1.000000
2,quesadillas,7.279767,0.393749


Submit "recommendations.tsv" in Gradescope.

In [13]:
#save as tsv
recs_df.to_csv("recommendations.tsv", sep='\t', index=False)

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [15]:
file_content = "For this assignment, I was feeling a bit lost on LinUCB and used the following resources to clear up my confusion: https://chatgpt.com/share/69910f4f-6bb4-8007-9b81-4765820f3adb , https://arxiv.org/abs/1003.0146 , and https://www.youtube.com/watch?v=JmbEheH7gVw ."
with open('acknowledgments.txt', 'w') as f:
    f.write(file_content)

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.